In [24]:
import cv2
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

## Specify the model to be used
COCO and MPI are body pose estimation model. COCO has 18 points and MPI has 15 points as output.

HAND is hand keypoints estimation model. It has 22 points as output

Ensure that the model files are available in the folders.

In [6]:
MODE = "MPI"

# if MODE is "COCO":
#     protoFile = "pose/coco/pose_deploy_linevec.prototxt"
#     weightsFile = "pose/coco/pose_iter_440000.caffemodel"
#     nPoints = 18
#     POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

# elif MODE is "MPI" :
protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
    

#### Let us load an image with multiple people and check what the model sees

In [7]:
# image1 = cv2.imread("multiple.jpeg")
# frameWidth = image1.shape[1]
# frameHeight = image1.shape[0]
# threshold = 0.1

#### Load the network and pass the image through the network

In [8]:
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

inWidth = 368
inHeight = 368
# inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (inWidth, inHeight),
#                           (0, 0, 0), swapRB=False, crop=False)

# net.setInput(inpBlob)
# output = net.forward()
# H = output.shape[2]
# W = output.shape[3]
# print(output.shape)

#### Slice a probability map from the output for a specific keypoint and plot the heatmap ( after resizing ) on the image itself

In [9]:
# i = 5
# probMap = output[0, i, :, :]
# probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
# plt.figure(figsize=[14,10])
# plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
# plt.imshow(probMap, alpha=0.6)
# plt.colorbar()
# plt.axis("off")

#### Similarly plot the affinity map on the image

In [10]:
# i = 24
# probMap = output[0, i, :, :]
# probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
# plt.figure(figsize=[14,10])
# plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
# plt.imshow(probMap, alpha=0.6)
# plt.colorbar()
# plt.axis("off")

#### Next, we find the keypoints for a image with only single person

In [11]:
counter = -1
df_is_empty = True
# for filename in os.listdir('.'):
for file_number in np.arange(100):
    filename = f"image_Roman10.MOV_{file_number}.jpg"
    if filename.endswith(".jpg"): 
#         try:
        counter += 1
        frame = cv2.imread(filename)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1
        is_None = False
        
#         point_df = []
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)

        output = net.forward()
        H = output.shape[2]
        W = output.shape[3]

        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

                # Add the point to the list if the probability is greater than the threshold
                points.append((int(x), int(y)))
            else :
                points.append(None)
                is_None = True
              
    flat_array = []
    for point in points:
        if point is None:
            flat_array.append(None)
            flat_array.append(None)
        else:
            for feature in point:
                flat_array.append(feature)
    flat_array = pd.Series(np.array(flat_array))
    # flat_array = np.array([feature for point in points for feature in point])
    if df_is_empty:
        df = pd.DataFrame([flat_array])
        df_is_empty = False
    else:
        df = df.append(flat_array, ignore_index=True)
    print('dataframe size', len(df))
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        # print(points)
    # cv2.putText(frame, "time taken = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8,
    #             (255, 50, 0), 2, lineType=cv2.LINE_AA)
    # cv2.putText(frame, "OpenPose using OpenCV", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    # cv2.imshow('Output-Keypoints', frameCopy)
#     cv2.imshow('Output-Skeleton', frame)
#     vid_writer.write(frame)
#     if hasFrame == False:
#         break
# vid_writer.release()
  
#         if is_None:
#             continue
#         flat_point = [e for l in points for e in l]
# #         print(flat_point)
#         flat_array = np.array([e for l in points for e in l])/400
#         point_dict = {i:flat_array[i] for i in np.arange(len(flat_array))}

#         if filename.find('right') >= 0:
#             point_dict['label'] = 1
#         else:
#             point_dict['label'] = 0
# #         if len(point_df) == 0:
#             point_df = pd.DataFrame(point_dict, index=[0])
#         else:
#             point_df = point_df.append(point_dict, ignore_index=True)

#         flat_array = flat_array / flat_array.max()
#         print(f"{flat_array} - {flat_array.max()}\n\n")
        # Draw Skeleton
        
#         if df_is_empty:
#             point_df = pd.DataFrame(point_dict, index=[0])
#             df_is_empty = False
#         else:
#             point_df = point_df.append(point_dict, ignore_index=True)
            
#         for pair in POSE_PAIRS:
#             partA = pair[0]
#             partB = pair[1]

#             if points[partA] and points[partB]:
#                 cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)
#         ##points
        
# #         plt.figure(figsize=[10,10])
# #         plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
#         print(counter)
#     print(filename)
        
#         except:
#             print(filename)
#             continue
            

        ##lines
#         plt.figure(figsize=[10,10])
#         plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

dataframe size 1
dataframe size 2
dataframe size 3
dataframe size 4
dataframe size 5
dataframe size 6
dataframe size 7
dataframe size 8
dataframe size 9
dataframe size 10
dataframe size 11
dataframe size 12
dataframe size 13
dataframe size 14
dataframe size 15
dataframe size 16
dataframe size 17
dataframe size 18
dataframe size 19
dataframe size 20
dataframe size 21
dataframe size 22
dataframe size 23
dataframe size 24
dataframe size 25
dataframe size 26
dataframe size 27
dataframe size 28
dataframe size 29
dataframe size 30
dataframe size 31
dataframe size 32
dataframe size 33
dataframe size 34
dataframe size 35
dataframe size 36
dataframe size 37
dataframe size 38
dataframe size 39
dataframe size 40
dataframe size 41
dataframe size 42
dataframe size 43
dataframe size 44
dataframe size 45
dataframe size 46
dataframe size 47
dataframe size 48
dataframe size 49
dataframe size 50
dataframe size 51
dataframe size 52
dataframe size 53
dataframe size 54
dataframe size 55
dataframe size 56
d

# change None to NaN

In [ ]:
import cv2
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [170]:
df2 = df.copy()
df2.fillna(value=np.nan, inplace=True)

## list NaNs tuples

In [171]:
# display(df.apply(np.nanmean, axis=1))
def nans_list(df):
    import math
    nans = []
    for i in np.arange(df.shape[0]):
        for j in np.arange(df.shape[1]):
            if math.isnan(df.iloc[i,j]):
                nans.append((i,j))
    return nans

In [174]:
nans = nans_list(df2)
print(f"NaNs left to deal with: {len(nans)}")

NaNs left to deal with: 840


## for points not at the edge, if NaN take the mean around them

In [175]:
persistent_nans = []
for point in nans:
    if point[0]>4 and point[0]<df2.shape[0]-5:
        df2.iloc[point[0], point[1]] = np.nanmean(df2.iloc[point[0]-5:point[0]+6,point[1] ])

print(f"NaNs left to deal with at the edges: {len(nans_list(df2))}")

NaNs left to deal with at the edges: 140


/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [176]:
display(nans_list(df2))

[(0, 0),
 (0, 1),
 (0, 10),
 (0, 11),
 (0, 14),
 (0, 15),
 (0, 28),
 (0, 29),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 28),
 (1, 29),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 10),
 (2, 11),
 (2, 28),
 (2, 29),
 (3, 0),
 (3, 1),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 14),
 (3, 15),
 (3, 28),
 (3, 29),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 14),
 (4, 15),
 (4, 28),
 (4, 29),
 (5, 0),
 (5, 1),
 (6, 0),
 (6, 1),
 (7, 0),
 (7, 1),
 (8, 0),
 (8, 1),
 (9, 0),
 (9, 1),
 (10, 0),
 (10, 1),
 (95, 0),
 (95, 1),
 (95, 2),
 (95, 3),
 (95, 10),
 (95, 11),
 (95, 12),
 (95, 13),
 (95, 20),
 (95, 21),
 (95, 26),
 (95, 27),
 (96, 0),
 (96, 1),
 (96, 6),
 (96, 7),
 (96, 8),
 (96, 9),
 (96, 10),
 (96, 11),
 (96, 14),
 (96, 15),
 (96, 18),
 (96, 19),
 (96, 20),
 (96, 21),
 (96, 24),
 (96, 25),
 (97, 0),
 (97, 1),
 (97, 4),
 (97, 5),
 (97, 6),
 (97, 7),
 (97, 8),
 (97, 

In [177]:
reverse_top = np.arange(0,15)[::-1]

In [178]:
for row in reverse_top:
    for col in np.arange(df2.shape[1]):
        if math.isnan(df2.iloc[row, col]):
            df2.iloc[row, col] = df2.iloc[row+1, col]

    

In [179]:
for row in np.arange(df2.shape[0]-10, df2.shape[0]):
    for col in np.arange(df2.shape[1]):
        if math.isnan(df2.iloc[row, col]):
            df2.iloc[row, col] = df2.iloc[row-1, col]


In [180]:
print(f"NaNs left to deal with: {len(nans_list(df2))}")

NaNs left to deal with: 0


## Anomaly detection and correction

In [181]:

def anomaly_detector(arr):
    array_mean = np.mean(arr)
    array_std = np.std(arr)
    mid_point = arr.iloc[int((arr.shape[0]+1)/2 - 1)]
    if array_std == 0 and np.abs(mid_point-array_mean)/array_std > 2:
#         print('anomaly handled')
        return array_mean, True
    return mid_point, False
    
   

In [182]:
iterations = 0
found_anomalies = True
while found_anomalies:
    iterations += 1
    found_anomalies = False
    for row in np.arange(5,df2.shape[0]-5):
        for col in np.arange(df2.shape[1]):
            result = anomaly_detector(df2.iloc[row-5:row+6, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
print(f"{iterations} iterations required")       

/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


21 iterations required


## anomalies on edges (top/bottom 5 rows)

In [183]:
def edge_anomaly_detector(arr, val):
    array_mean = np.mean(arr)
    array_std = np.std(arr)
#     mid_point = arr.iloc[int((arr.shape[0]+1)/2 - 1)]
    if np.abs(val-array_mean)/array_std > 2:
#         print('anomaly handled')
        return array_mean, True
    return val, False
    
   

In [184]:
iterations = 0
found_anomalies = True
while found_anomalies:
    iterations += 1
    found_anomalies = False
    for row in np.arange(5):
        for col in np.arange(df2.shape[1]):
            result = edge_anomaly_detector(df2.iloc[:5, col], df2.iloc[row, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
    for row in np.arange(df2.shape[0]):
        for col in np.arange(df2.shape[1]):
            result = edge_anomaly_detector(df2.iloc[df2.shape[0]-5:, col], df2.iloc[row, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
print(f"{iterations} iterations required")    

/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


5 iterations required


In [185]:
display(df2)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1139.244000,211.000000,969.950000,469.000000,834.000000,352.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,540.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
1,1139.244000,211.000000,969.950000,493.000000,834.000000,493.000000,1126.0,305.0,1226.800000,516.000,...,1095.588479,982.963965,918.0,563.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
2,1139.244000,211.000000,969.950000,493.000000,850.800000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,793.0,493.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
3,1139.244000,211.000000,969.950000,493.000000,834.000000,375.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,563.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
4,1139.244000,211.000000,969.950000,481.000000,834.000000,352.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,540.0,1043.0,722.4,1113.094748,941.226804,993.200000,455.200000
5,1139.244000,211.000000,969.950000,481.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1095.588479,986.000000,793.0,399.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
6,1139.244000,211.000000,969.950000,487.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1047.300000,1004.600000,793.0,375.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
7,1139.244000,211.000000,969.950000,487.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,834.0,493.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
8,1139.244000,211.000000,969.950000,487.000000,960.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,516.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
9,1139.244000,211.000000,969.950000,485.500000,936.555556,80.333333,1126.0,305.0,1226.800000,514.872,...,1095.588479,970.819824,834.0,469.0,1043.0,722.4,1113.094748,950.134022,993.200000,257.317708


In [187]:
x = 2
y = 0
if y>0 and x/y >0:
    print('yes')
else:
    print('no')

no
